In [14]:
import os
import json
import pprint

pp = pprint.PrettyPrinter(indent=2)

directory = os.fsencode("../synthea-1")
count = 0

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".json"):
        count += 1
        if count == 50:
            with open(os.path.join("../synthea-1", filename)) as f:
                data = json.load(f)
                pp.pprint(data)
        


{ 'entry': [ { 'fullUrl': 'urn:uuid:07831da0-5d36-4898-8f87-09f10c2c9afc',
               'request': {'method': 'POST', 'url': 'Patient'},
               'resource': { 'address': [ { 'city': 'Greenfield',
                                            'country': 'US',
                                            'extension': [ { 'extension': [ { 'url': 'latitude',
                                                                              'valueDecimal': 42.59696558997381},
                                                                            { 'url': 'longitude',
                                                                              'valueDecimal': -72.63080270908928}],
                                                             'url': 'http://hl7.org/fhir/StructureDefinition/geolocation'}],
                                            'line': ['661 Runolfsson Path'],
                                            'postalCode': '01301',
                                        

                             'resourceType': 'Observation',
                             'status': 'final',
                             'subject': { 'reference': 'urn:uuid:07831da0-5d36-4898-8f87-09f10c2c9afc'},
                             'valueQuantity': { 'code': '/min',
                                                'system': 'http://unitsofmeasure.org',
                                                'unit': '/min',
                                                'value': 13}}},
             { 'fullUrl': 'urn:uuid:23b95a94-395a-4fc0-98b0-5aefd1eeb4fc',
               'request': {'method': 'POST', 'url': 'Observation'},
               'resource': { 'category': [ { 'coding': [ { 'code': 'survey',
                                                           'display': 'survey',
                                                           'system': 'http://terminology.hl7.org/CodeSystem/observation-category'}]}],
                             'code': { 'coding': [ { 'code': '72166-2',
 

                             'telecom': [ { 'system': 'phone',
                                            'value': '4137730211'}],
                             'type': [ { 'coding': [ { 'code': 'prov',
                                                       'display': 'Healthcare '
                                                                  'Provider',
                                                       'system': 'http://terminology.hl7.org/CodeSystem/organization-type'}],
                                         'text': 'Healthcare Provider'}]}},
             { 'fullUrl': 'urn:uuid:f13236fd-65e1-303e-a8f1-9f018abcd6eb',
               'request': {'method': 'POST', 'url': 'Practitioner'},
               'resource': { 'active': True,
                             'address': [ { 'city': 'GREENFIELD',
                                            'country': 'US',
                                            'line': ['164 HIGH STREET'],
                                            'pos

                             'presentedForm': [ { 'contentType': 'text/plain',
                                                  'data': 'CjIwMTAtMTAtMjcKCiMgQ2hpZWYgQ29tcGxhaW50Ck5vIGNvbXBsYWludHMuCgojIEhpc3Rvcnkgb2YgUHJlc2VudCBJbGxuZXNzCkNoZXllbm5lMTY5IGlzIGEgMTAgeWVhci1vbGQgbm9uLWhpc3BhbmljIHdoaXRlIGZlbWFsZS4KCiMgU29jaWFsIEhpc3RvcnkKIFBhdGllbnQgaGFzIG5ldmVyIHNtb2tlZC4KClBhdGllbnQgY29tZXMgZnJvbSBhIGhpZ2ggc29jaW9lY29ub21pYyBiYWNrZ3JvdW5kLiBQYXRpZW50IGN1cnJlbnRseSBoYXMgTk8gSU5TVVJBTkNFLgoKIyBBbGxlcmdpZXMKTm8gS25vd24gQWxsZXJnaWVzLgoKIyBNZWRpY2F0aW9ucwpObyBBY3RpdmUgTWVkaWNhdGlvbnMuCgojIEFzc2Vzc21lbnQgYW5kIFBsYW4KUGF0aWVudCBpcyBwcmVzZW50aW5nIHdpdGggYWN1dGUgYnJvbmNoaXRpcyAoZGlzb3JkZXIpLiAKCiMjIFBsYW4KClRoZSBmb2xsb3dpbmcgcHJvY2VkdXJlcyB3ZXJlIGNvbmR1Y3RlZDoKLSBzcHV0dW0gZXhhbWluYXRpb24gKHByb2NlZHVyZSkKVGhlIHBhdGllbnQgd2FzIHByZXNjcmliZWQgdGhlIGZvbGxvd2luZyBtZWRpY2F0aW9uczoKLSBhY2V0YW1pbm9waGVuIDMyNSBtZyBvcmFsIHRhYmxldApUaGUgcGF0aWVudCB3YXMgcGxhY2VkIG9uIGEgY2FyZXBsYW46Ci0gcmVzcGlyYXRvcnkgdGhlcm

                             'subject': { 'reference': 'urn:uuid:07831da0-5d36-4898-8f87-09f10c2c9afc'},
                             'valueQuantity': { 'code': '10*3/uL',
                                                'system': 'http://unitsofmeasure.org',
                                                'unit': '10*3/uL',
                                                'value': 10.302}}},
             { 'fullUrl': 'urn:uuid:976b3372-280d-44d8-981a-79f86929e99f',
               'request': {'method': 'POST', 'url': 'Observation'},
               'resource': { 'category': [ { 'coding': [ { 'code': 'laboratory',
                                                           'display': 'laboratory',
                                                           'system': 'http://terminology.hl7.org/CodeSystem/observation-category'}]}],
                             'code': { 'coding': [ { 'code': '789-8',
                                                     'display': 'Erythrocytes '
             

...

             { 'fullUrl': 'urn:uuid:2dd8bf6e-1e6c-4461-97d4-0b4f96fc4157',
               'request': {'method': 'POST', 'url': 'Observation'},
               'resource': { 'category': [ { 'coding': [ { 'code': 'procedure',
                                                           'display': 'procedure',
                                                           'system': 'http://terminology.hl7.org/CodeSystem/observation-category'}]}],
                             'code': { 'coding': [ { 'code': '59557-9',
                                                     'display': 'Treatment '
                                                                'status Cancer',
                                                     'system': 'http://loinc.org'}],
                                       'text': 'Treatment status Cancer'},
                             'effectiveDateTime': '2018-10-17T09:55:34-04:00',
                             'encounter': { 'reference': 'urn:uuid:ac35cb28-4fad-46d0-83fc

                                            'format': { 'code': 'urn:ihe:iti:xds:2017:mimeTypeSufficient',
                                                        'display': 'mimeType '
                                                                   'Sufficient',
                                                        'system': 'http://ihe.net/fhir/ValueSet/IHE.FormatCode.codesystem'}}],
                             'context': { 'encounter': [ { 'reference': 'urn:uuid:64c1e334-9ee0-4eb2-9f29-2674b6a0beab'}],
                                          'period': { 'end': '2019-06-12T10:10:34-04:00',
                                                      'start': '2019-06-12T09:55:34-04:00'}},
                             'custodian': { 'display': 'BAYSTATE FRANKLIN '
                                                       'MEDICAL CENTER',
                                            'reference': 'urn:uuid:fbf6180e-b800-3ebe-b91d-93d0288c400e'},
                             'date': '2019-

                             'subject': { 'reference': 'urn:uuid:07831da0-5d36-4898-8f87-09f10c2c9afc'},
                             'valueCodeableConcept': { 'coding': [ { 'code': '266919005',
                                                                     'display': 'Never '
                                                                                'smoked '
                                                                                'tobacco '
                                                                                '(finding)',
                                                                     'system': 'http://snomed.info/sct'}],
                                                       'text': 'Never smoked '
                                                               'tobacco '
                                                               '(finding)'}}},
             { 'fullUrl': 'urn:uuid:df9e06f8-78f3-4879-9c92-f0204f79857c',
               'request': {'

                             'category': [ { 'coding': [ { 'code': 'clinical-note',
                                                           'display': 'Clinical '
                                                                      'Note',
                                                           'system': 'http://hl7.org/fhir/us/core/CodeSystem/us-core-documentreference-category'}]}],
                             'content': [ { 'attachment': { 'contentType': 'text/plain',
                                                            'data': 'CjIwMTktMTAtMDIKCiMgQ2hpZWYgQ29tcGxhaW50Ck5vIGNvbXBsYWludHMuCgojIEhpc3Rvcnkgb2YgUHJlc2VudCBJbGxuZXNzCkNoZXllbm5lMTY5IGlzIGEgMTkgeWVhci1vbGQgbm9uLWhpc3BhbmljIHdoaXRlIGZlbWFsZS4gUGF0aWVudCBoYXMgYSBoaXN0b3J5IG9mIGFjdXRlIGJyb25jaGl0aXMgKGRpc29yZGVyKSwgYWN1dGUgYmFjdGVyaWFsIHNpbnVzaXRpcyAoZGlzb3JkZXIpLCBmcmFjdHVyZSBvZiByaWIsIHZpcmFsIHNpbnVzaXRpcyAoZGlzb3JkZXIpLgoKIyBTb2NpYWwgSGlzdG9yeQogUGF0aWVudCBpcyBhbiBhY3RpdmUgc21va2VyLiBQYXRpZW50IGlkZW50aWZpZXMg

               'resource': { 'author': [ { 'display': 'Dr. Jesse626 Hegmann834',
                                           'reference': 'urn:uuid:f13236fd-65e1-303e-a8f1-9f018abcd6eb'}],
                             'category': [ { 'coding': [ { 'code': 'clinical-note',
                                                           'display': 'Clinical '
                                                                      'Note',
                                                           'system': 'http://hl7.org/fhir/us/core/CodeSystem/us-core-documentreference-category'}]}],
                             'content': [ { 'attachment': { 'contentType': 'text/plain',
                                                            'data': 'CjIwMjAtMDEtMTUKCiMgQ2hpZWYgQ29tcGxhaW50Ck5vIGNvbXBsYWludHMuCgojIEhpc3Rvcnkgb2YgUHJlc2VudCBJbGxuZXNzCkNoZXllbm5lMTY5IGlzIGEgMTkgeWVhci1vbGQgbm9uLWhpc3BhbmljIHdoaXRlIGZlbWFsZS4gUGF0aWVudCBoYXMgYSBoaXN0b3J5IG9mIGFjdXRlIGJyb25jaGl0aXMgKGRpc29yZGVyKSwgYWN1dGUgYmFj